In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
""" if(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu" """
device ="cpu"

In [30]:
data1=pd.read_csv("data/finbadtrb.csv")
data2=pd.read_csv("data/finbadtrg.csv")

data3=pd.read_csv("data/finbadvalb.csv")
data4=pd.read_csv("data/finbadvalg.csv")

data5=pd.read_csv("data/fingoodtrg.csv")
data6=pd.read_csv("data/fingoodvalg.csv")

data = pd.concat([data1,data2,data3,data4,data5,data6])
params=len(data.columns)-1

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import torch.utils
from pickle import dump
encoder = LabelEncoder()
data["Label"] = encoder.fit_transform(data["Label"])
X=data.drop(columns="Label").to_numpy(dtype=np.float32)
y=data["Label"].to_numpy(dtype=np.float32)
#scaler=MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

train_dataset= torch.utils.data.TensorDataset(torch.tensor(X_train),torch.tensor(y_train))

test_dataset= torch.utils.data.TensorDataset(torch.tensor(X_test),torch.tensor(y_test))
#dump(scaler, open('scaler.pkl', 'wb'))

train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=16,num_workers=3)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=4,num_workers=3)

In [32]:

class slouch_detection(nn.Module):
    def __init__(self):
        super(slouch_detection,self).__init__()
        self.layers = nn.Sequential(
            nn.LayerNorm(params),
            nn.Linear(params,64),
            nn.SiLU(),
            nn.Dropout(0.2),
            nn.LayerNorm(64),
            nn.Linear(64,1),
            nn.Sigmoid()
        )
    @torch.compile(fullgraph=True, dynamic=True, options = {"epilogue_fusion": True, "max_autotune": True,
                    "shape_padding": True})
    def forward(self,x):
        return self.layers(x)


In [33]:
model = slouch_detection().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Assuming you're using Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=2e-2)  # You can adjust the learning rate as needed


In [34]:
total=0
for params in model.parameters():
    total+=params.numel()
total

1511

In [35]:
def train_model(model, train_loader, optimizer, criterion, epochs,device,best_acc):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            #print(outputs.size(),labels.unsqueeze(dim=1).size())
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            
            # Calculate accuracy
            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            print()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")
        test_model(model, test_dataloader, criterion,device,best_acc)

# Testing loop
def test_model(model, test_loader, criterion,device,best_acc):
    model.eval()
    with torch.inference_mode():
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        test_loss = running_loss / len(test_loader.dataset)
        test_acc = correct / total
        if(best_acc<test_acc):
            torch.save(model.state_dict(),"slouch_detector.pt")
            best_acc=test_acc
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [36]:
best_acc=0
train_model(model, train_dataloader, optimizer, criterion,device=device, epochs=4,best_acc=best_acc)  # Adjust epochs as needed

test_model(model, test_dataloader, criterion,device,best_acc)














































































Epoch 1/4, Loss: 0.6006, Accuracy: 0.6876
Test Loss: 0.5704, Test Accuracy: 0.6733













































































Epoch 2/4, Loss: 0.5167, Accuracy: 0.7258
Test Loss: 0.4344, Test Accuracy: 0.7822













































































Epoch 3/4, Loss: 0.3405, Accuracy: 0.8332
Test Loss: 0.3336, Test Accuracy: 0.8465













































































Epoch 4/4, Loss: 0.3111, Accuracy: 0.8552
Test Loss: 0.2723, Test Accuracy: 0.8861
Test Loss: 0.2723, Test Accuracy: 0.8861


In [37]:
# test_model(model, test_dataloader, criterion,device,best_acc)